# #08. Метод классификации K_NN

Загрузим данные

In [1]:
import numpy as np
from sklearn.datasets import load_digits

digits_dataset = load_digits()
x = digits_dataset.data
y = digits_dataset.target
print(x)
print(y)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
[0 1 2 ... 8 9 8]


Разделим данные на обучающую и тестовую выборки

In [2]:
from math import floor

train_size = floor(x.shape[0] * 0.8)
train_size

1437

In [3]:
x_train = x[:train_size]
x_test = x[train_size:]

y_train = y[:train_size]
y_test = y[train_size:]

Создадим функции расстояний

- distance определяет расстояние между 2 точками
- distances определяет расстояние от всех точек из массива точек (2-мерный массив, где строки – точки, столбцы – координаты) до одной точки

In [4]:
# noinspection PyShadowingNames
def distance(x: np.ndarray, y: np.ndarray):
    return np.sqrt(np.sum((x - y) ** 2))


# noinspection PyShadowingNames
def distances(x: np.ndarray, y: np.ndarray):
    return np.sqrt(np.sum((x - y) ** 2, axis=1))

In [5]:
distances(x, x[0])

array([ 0.        , 59.55669568, 54.12947441, ..., 50.37856687,
       37.06750599, 47.03190407])

In [6]:
distance(x[1], x[0])

59.55669567731239

Создадим классификатор по методу k ближайших соседей

In [7]:
import statistics


# noinspection PyPep8Naming
class K_NN:
    def __init__(self, n_neighbors=3):
        self.n_neighbors = n_neighbors
        self.labels: np.ndarray = None
        self.dataset: np.ndarray = None


    # noinspection PyShadowingNames
    def fit(self, x: np.ndarray, y: np.ndarray):
        self.dataset = x
        self.labels = y


    def _get_closest_points(self, point):
        distances_to_point: np.ndarray = distances(self.dataset, point)
        closest_points = distances_to_point.argpartition(self.n_neighbors - 1)[:self.n_neighbors]
        return closest_points


    def predict(self, points: np.ndarray):
        result = np.empty(points.shape[0])
        for i, point in enumerate(points):
            closest_points = self._get_closest_points(point)
            labels = self.labels[closest_points]
            result[i] = statistics.mode(labels)
        return result


    def get_neighbors(self, points: np.ndarray):
        result = np.empty((points.shape[0], self.n_neighbors))
        for i, point in enumerate(points):
            result[i] = self._get_closest_points(point)
        return result

Обучим созданный нами классификатор и определим accuracy на тестовой выборке

In [8]:
knn = K_NN(n_neighbors=3)
knn.fit(x_train, y_train)
knn_prediction = knn.predict(x_test)

In [9]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, knn_prediction)

0.9611111111111111

Для сравнения обучим аналогичный классификатор из sklearn и определим accuracy

In [10]:
from sklearn.neighbors import KNeighborsClassifier

knn_sklearn = KNeighborsClassifier(n_neighbors=3)
knn_sklearn.fit(x_train, y_train)
knn_sklearn_prediction = knn_sklearn.predict(x_test)
accuracy_score(y_test, knn_sklearn_prediction)

0.9666666666666667

Получились разные результаты. Выясним, почему это произошло

С помощью обоих классификаторов найдем для каждой точки из тестовой выборки ее соседей

In [11]:
knn_neighbors = knn.get_neighbors(x_test)
knn_neighbors

array([[1417.,  986.,  833.],
       [ 865.,  259., 1418.],
       [ 613.,  520.,  840.],
       ...,
       [ 148.,  248.,  242.],
       [ 254.,  251.,  849.],
       [ 183.,  248., 1015.]])

In [12]:
knn_sklearn_neighbors = knn_sklearn.kneighbors(x_test, return_distance=False)
knn_sklearn_neighbors

array([[1417,  986,  833],
       [ 865,  259, 1418],
       [ 613,  520,  840],
       ...,
       [ 148,  248,  242],
       [ 254,  251,  849],
       [ 183,  248, 1015]], dtype=int64)

Найденные соседи отличаются только для 1 точки

In [13]:
are_rows_same = (np.sort(knn_neighbors, axis=1) == np.sort(knn_sklearn_neighbors, axis=1)).all(axis=1)

In [14]:
np.where(~are_rows_same)

(array([290], dtype=int64),)

Результаты для этой точки отличаются одним соседом

In [15]:
knn_neighbors[~are_rows_same]

array([[114., 759., 699.]])

In [16]:
knn_sklearn_neighbors[~are_rows_same]

array([[114, 759, 761]], dtype=int64)

При этом расстояния от точки до обоих вариантов соседа одинаковы. Это значит, что результаты обоих классификаторов равнозначны

In [17]:
distance(x[train_size + 290], x[699])

33.075670817082454

In [18]:
distance(x[train_size + 290], x[761])

33.075670817082454